In [1]:
# Cell 1: A Clean and Simple Installation
!pip install -q -U "sentence-transformers" "faiss-cpu" "pymupdf" "langchain" "google-generativeai" "ipywidgets"


In [2]:
import os
import numpy as np
import faiss
import google.generativeai as genai
import fitz  # PyMuPDF
from google.colab import files, userdata
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter
import ipywidgets as widgets
from IPython.display import display, clear_output

print("✅ All libraries installed and imported successfully!")

✅ All libraries installed and imported successfully!


In [3]:
import google.generativeai as genai
from google.colab import userdata

# Make sure your API key is configured
try:
    genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))
except userdata.SecretNotFoundError:
    print("🛑 Error: 'GOOGLE_API_KEY' not found in Colab secrets.")
except Exception as e:
    print(f"An error occurred: {e}")

print("--- Available Models ---")

# List all available models
for model in genai.list_models():
  # Check if the 'generateContent' method is supported
  if 'generateContent' in model.supported_generation_methods:
    print(f"✅ Model Name: {model.name}")
    print(f"   Supported Methods: {model.supported_generation_methods}\n")

--- Available Models ---
✅ Model Name: models/gemini-2.5-pro-preview-03-25
   Supported Methods: ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']

✅ Model Name: models/gemini-2.5-flash-preview-05-20
   Supported Methods: ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']

✅ Model Name: models/gemini-2.5-flash
   Supported Methods: ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']

✅ Model Name: models/gemini-2.5-flash-lite-preview-06-17
   Supported Methods: ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']

✅ Model Name: models/gemini-2.5-pro-preview-05-06
   Supported Methods: ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']

✅ Model Name: models/gemini-2.5-pro-preview-06-05
   Supported Methods: ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']

✅ Model Name: models/gemini-2.5-pro
   Supported

In [4]:
# Cell 2: API Key Configuration (Updated with 'latest' model)
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=GOOGLE_API_KEY)
    print("🔑 API Key configured successfully!")

    # Use the latest stable Flash model
    model_gen = genai.GenerativeModel('gemini-flash-latest')

except userdata.SecretNotFoundError:
    print("🛑 Error: 'GOOGLE_API_KEY' not found. Please follow the instructions to add it to Colab secrets.")
except Exception as e:
    print(f"An error occurred: {e}")

🔑 API Key configured successfully!


In [5]:
# Cell 3: Upload and Process PDF (Corrected)

# --- Global Variables ---
text_chunks = []
faiss_index = None
# Corrected the model name below
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
d = embedding_model.get_sentence_embedding_dimension()

# --- Upload and Process Logic ---
print("Please upload your PDF file.")
uploaded = files.upload()

if not uploaded:
    print("\nNo file uploaded. Please run the cell again.")
else:
    file_name = next(iter(uploaded))
    print(f"\n📄 Processing '{file_name}'...")

    # 1. Extract text from PDF
    doc = fitz.open(file_name)
    full_text = "".join(page.get_text() for page in doc)
    doc.close()
    print(f"   - Extracted {len(full_text)} characters.")

    # 2. Split text into chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    text_chunks = text_splitter.split_text(full_text)
    print(f"   - Split into {len(text_chunks)} chunks.")

    # 3. Create embeddings and FAISS index
    embeddings = embedding_model.encode(text_chunks, show_progress_bar=True)
    faiss_index = faiss.IndexFlatL2(d)
    faiss_index.add(np.array(embeddings).astype('float32'))
    print("✅ Index created successfully. You can now chat with the document below.")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Please upload your PDF file.


Saving Surrounded by Idiots - Thomas Erikson.pdf to Surrounded by Idiots - Thomas Erikson (2).pdf

📄 Processing 'Surrounded by Idiots - Thomas Erikson (2).pdf'...
   - Extracted 452288 characters.
   - Split into 502 chunks.


Batches:   0%|          | 0/16 [00:00<?, ?it/s]

✅ Index created successfully. You can now chat with the document below.


In [6]:
from google.colab import output
output.enable_custom_widget_manager()

In [7]:
from google.colab import output
output.disable_custom_widget_manager()

In [8]:
# Cell 4: Ask a Question Directly (No Chatbot)

def get_rag_response(query):
    """Simplified RAG function that takes a string query."""
    if not faiss_index or not text_chunks:
        return "Error: Document not processed. Please run the cells above first."

    # 1. Embed query and search index
    query_vec = embedding_model.encode([query]).astype('float32')
    distances, indices = faiss_index.search(query_vec, k=5)
    retrieved_chunks = [text_chunks[i] for i in indices[0]]
    context = "\n\n---\n\n".join(retrieved_chunks)

    # 2. Create prompt and get model response
    prompt = f"""
    Answer the user's question based ONLY on the following context:
    CONTEXT:
    {context}

    QUESTION:
    {query}
    """
    try:
        response = model_gen.generate_content(prompt)
        return response.text
    except Exception as e:
        return f"An error occurred: {e}"

# --- Ask your question here ---
# Just change the text in this variable and run the cell again.
my_question = "who is the author and what to do when surrounded by idiots?"

# --- Get and print the response ---
print("🧠 Thinking...")
answer = get_rag_response(my_question)

print(f"\n👤 Your Question: {my_question}")
print("---------------------------------")
print(f"🤖 AI's Answer:\n{answer}")

🧠 Thinking...


ERROR:tornado.access:503 POST /v1beta/models/gemini-flash-latest:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 5075.01ms



👤 Your Question: who is the author and what to do when surrounded by idiots?
---------------------------------
🤖 AI's Answer:
The author is **Thomas Erikson**.

Regarding what to do when surrounded by people who seem like idiots, the context advises that you should understand that they "are, in fact, not idiots at all. Instead, they are individuals worthy of respect, understanding, and being valued."

The book's goal is to provide practical advice in communicating effectively with the different types (Red, Yellow, Green, and Blue). If you don’t understand and use the frameworks and principles outlined in the book, you will "continue to be surrounded by idiots."
